Ensure you login to Azure az login - device

Ensure you change the Kernel to Python 3.10 AzureML

In [ ]:
pip install python-dotenv

In [ ]:
pip install datasets -U

In [ ]:
from datasets import load_dataset
from abc import ABC

In [ ]:
class InputDataset(ABC):
    def __init__(self):
        super().__init__()
        (
            self.train_data_file_name,
            self.test_data_file_name,
            self.eval_data_file_name,
        ) = (None, None, None)

In [ ]:
class CQnAHuggingFaceInputDataset(InputDataset):
    """
    Loads the HuggingFace dataset
    """

    def __init__(self):
        super().__init__()

    def load_hf_dataset(
        self,
        dataset_name,
        train_sample_size=10,
        val_sample_size=10,
        test_sample_size=10,
        train_split_name="train",
        val_split_name="validation",
        test_split_name="test",
    ):
        full_dataset = load_dataset(dataset_name)

        if val_split_name is not None:
            train_data = full_dataset[train_split_name].select(range(train_sample_size))
            val_data = full_dataset[val_split_name].select(range(val_sample_size))
            test_data = full_dataset[test_split_name].select(range(test_sample_size))
        else:
            train_val_data = full_dataset[train_split_name].select(
                range(train_sample_size + val_sample_size)
            )
            train_data = train_val_data.select(range(train_sample_size))
            val_data = train_val_data.select(
                range(train_sample_size, train_sample_size + val_sample_size)
            )
            test_data = full_dataset[test_split_name].select(range(test_sample_size))

        return train_data, val_data, test_data

In [ ]:
# We can define train and test sample sizes here. Validation size is kept same as test sample size
train_sample_size = 100
val_sample_size = 100

# Sample notebook using the dataset: https://huggingface.co/datasets/tau/commonsense_qa
dataset_name = "tau/commonsense_qa"
input_dataset = CQnAHuggingFaceInputDataset()

# Note: train_split_name and test_split_name can vary by dataset. They are passed as arguments in load_hf_dataset.
# If validation_split_name is None, the below function will split the train set to create the specified sized validation set.
train, val, _ = input_dataset.load_hf_dataset(
    dataset_name=dataset_name,
    train_sample_size=train_sample_size,
    val_sample_size=val_sample_size,
    train_split_name="train",
    val_split_name="validation",
)

print("Len of train data sample is " + str(len(train)))
print("Len of validation data sample is " + str(len(val)))

In [ ]:
! mkdir -p data

In [ ]:
train_data_path = "data/train_original_data.jsonl"

In [ ]:
import json

In [ ]:
system_prompt = "You are a helpful assistant. Your output should only be one of the five choices: 'A', 'B', 'C', 'D', or 'E'."
user_prompt_template = "Answer the following multiple-choice question by selecting the correct option.\n\nQuestion: {question}\nAnswer Choices:\n{answer_choices}"

for row in train:
    data = {"messages": []}
    data["messages"].append(
        {
            "role": "system",
            "content": system_prompt,
        }
    )
    question, choices = row["question"], row["choices"]
    labels, choice_list = choices["label"], choices["text"]
    answer_choices = [
        "({}) {}".format(labels[i], choice_list[i]) for i in range(len(labels))
    ]
    answer_choices = "\n".join(answer_choices)
    data["messages"].append(
        {
            "role": "user",
            "content": user_prompt_template.format(
                question=question, answer_choices=answer_choices
            ),
        }
    )
    with open(train_data_path, "a") as f:
        f.write(json.dumps(data) + "\n")

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
print("Loading environment variables from .env file...")
load_dotenv()

In [ ]:
# Get Azure AI Foundry credentials from environment variables
teacher_model_name = os.getenv('TEACHER_MODEL_NAME')
teacher_model_endpoint_url = os.getenv('TEACHER_MODEL_ENDPOINT')
teacher_model_api_key = os.getenv('TEACHER_MODEL_KEY')

# Print values for debugging (remove in production)
print(f"Teacher Model Name: {teacher_model_name}")
print(f"Teacher Model Endpoint: {teacher_model_endpoint_url}")
# Don't print the API key for security reasons
print(f"Teacher Model API Key loaded: {'Yes' if teacher_model_api_key else 'No'}")

In [ ]:
pip install azure-ai-inference

In [ ]:
import os
import json
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

In [ ]:
def process_question(question_data):
    try:
        messages = []
        for msg in question_data["messages"]:
            if msg["role"] == "system":
                messages.append(SystemMessage(content=msg["content"]))
            elif msg["role"] == "user":
                messages.append(UserMessage(content=msg["content"]))

        response = client.complete(
            messages=messages,
            model=model_name,
            max_tokens=100  # Reduced since we just need short answers like A, B, C, D, or E
        )

        return {
            "question": question_data["messages"][1]["content"],
            "response": response.choices[0].message.content,
            "full_response": response
        }
    except Exception as e:
        return {
            "question": question_data["messages"][1]["content"] if len(question_data["messages"]) > 1 else "Error",
            "response": f"Error: {str(e)}",
            "full_response": None
        }

In [ ]:
endpoint = teacher_model_endpoint_url
model_name = teacher_model_name
key = teacher_model_api_key
client = ChatCompletionsClient(endpoint=endpoint, credential=AzureKeyCredential(key))

In [ ]:
# Read the JSONL file and process each question
results = []
with open(train_data_path, 'r', encoding='utf-8') as file:
    print(f"Processing questions from {train_data_path}")
    for i, line in enumerate(file):
        if line.strip():  # Skip empty lines
            try:
                question_data = json.loads(line)
                print(f"Processing question {i+1}...")
                result = process_question(question_data)
                results.append(result)
                print(f"Question {i+1} response: {result['response']}")
            except json.JSONDecodeError as e:
                print(f"Error parsing line {i+1}: {str(e)}")
            except Exception as e:
                print(f"Error processing line {i+1}: {str(e)}")

In [ ]:
# Read the JSONL file and process each question
results = []
with open(train_data_path, 'r', encoding='utf-8') as file:
    print(f"Processing questions from {train_data_path}")
    for i, line in enumerate(file):
        if line.strip():  # Skip empty lines
            try:
                question_data = json.loads(line)
                print(f"Processing question {i+1}...")
                result = process_question(question_data)
                results.append(result)
                print(f"Question {i+1} response: {result['response']}")
            except json.JSONDecodeError as e:
                print(f"Error parsing line {i+1}: {str(e)}")
            except Exception as e:
                print(f"Error processing line {i+1}: {str(e)}")

In [ ]:
output_file_path = "./data/train_data.jsonl"
with open(output_file_path, 'w', encoding='utf-8') as f:
    for result in results:
        # Extract just the question content (removing the instruction part)
        question_text = result["question"]
        # if "Question: " in question_text:
        #     question_text = question_text.split("Question: ")[1].split("\nAnswer Choices:")[0]

        # Create the simplified output format
        output_line = {
            "Question": question_text,
            "Answer": result["response"]
        }

        # Write as JSONL (one JSON object per line)
        f.write(json.dumps(output_line, ensure_ascii=False) + '\n')